In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
if os.path.exists("model.pth"):
    model.load_state_dict(torch.load("model.pth"))
    print("Loaded model from model.pth")
else:
    print("No model found, creating new model")
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x.to(device))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
import os
import torch
from torch import nn
import sqlite3
import sys
import traceback
import numpy as np


#Connecting to the SQLite database
dbPath = r'Data\datasetNY.db'
dbConnection = sqlite3.connect(dbPath)

dbData = []

#Retrieve the data from the database
try:
    dbData = dbConnection.execute('SELECT * FROM Motor_Vehicle_Collisions_Crashes').fetchall()
except:
    print('Error retrieving the data from the database')
else:
    print('Successfully retrieved the data from the databse')

first = int(float(dbData[1][2]))

n_input = first
n_hidden = 15
n_out = 1
batch_size = 100
learning_rate = 0.01

data_x = torch.randn(batch_size, n_input)
data_y = (torch.rand(size=(batch_size, 1)) < 0.5).float()

print(data_x.size())
print(data_y.size())

model = nn.Sequential(nn.Linear(n_input, n_hidden),
                      nn.ReLU(),
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid())
print(model)

loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

losses = []
for epoch in range(5000):
    pred_y = model(data_x)
    loss = loss_function(pred_y, data_y)
    losses.append(loss.item())

    model.zero_grad()
    loss.backward()

    optimizer.step()
    
import matplotlib.pyplot as plt
plt.plot(losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title("Learning rate %f"%(learning_rate))
plt.show()

dbConnection.close

In [11]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datetime
import sqlite3
import sys
import traceback
import numpy as np
import Data.database_handler as dbHandler
from torchvision import transforms, utils

#Connecting to the SQLite database
dbPath = r'Data\datasetNY.db'

class AccidentDataset(Dataset):
    def __init__(self, db_path, transform=None):
        self.coordinates = dbHandler.get_n_data_datetime_converted(db_path, 20000)
        self.coordinates = pd.DataFrame(self.coordinates, columns=['datetime', 'latitude', 'longitude'])
        self.transform = transform

    def __len__(self):
        return len(self.coordinates)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        datetime = self.coordinates.iloc[idx, 0:1]
        datetime = np.array([datetime]).squeeze()
        datetime = datetime.astype('float')
        datetime = torch.from_numpy(datetime).float()
        #datetime = datetime.reshape(1, -1)

        coordinates = self.coordinates.iloc[idx, 1:3]
        coordinates = np.array([coordinates]).squeeze()
        coordinates = coordinates.astype('float')
        coordinates = torch.from_numpy(coordinates).float()
        sample = {'coordinates': coordinates, 'datetime' : datetime}

        if self.transform:
            sample = self.transform(sample)

        return coordinates, datetime


#     #Making constructor
#     def __init__(self, dbData, root_dir, transform_none):
#         self.coordinates = dbData
#         self.root_dir = root_dir
#         self.transform = transform_none
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# load data from database and split into train and test


accident_dataset = AccidentDataset(dbPath)

print(accident_dataset[0])
#print(accident_dataset[0]['coordinates'][0])
#print(accident_dataset[0]['datetime'])

#Create new array with 60% of the data
train_size = int(0.6 * len(accident_dataset))
test_size = len(accident_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(accident_dataset, [train_size, test_size])

print(len(train_dataset))
print(len(test_dataset))
print(len(accident_dataset))

#Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

#Creating Network class
class Network(nn.Module):

    #Making constructor
    def __init__(self, in_d=2, out_d=2, hidden_d=8, num_hidden=1):
        super(Network, self).__init__()
        #self.bn = nn.BatchNorm1d(input_size)
        #Creating fully connected layers
        #self.fclayer1 = nn.Linear(input_size, 50)
        #self.fclayer2 = nn.Linear(50, number_of_classes)
        self.rnn = nn.RNN(input_size=in_d, hidden_size=hidden_d, num_layers=num_hidden)
        self.fc = nn.Linear(hidden_d, out_d)
    
    #Creating forward method to do forward propagation
    def forward(self, x, h0):
        r, h = self.rnn(x, h0)
        y = self.fc(r)  # no activation on the output
        return y, h
    #def forward(self, input):
        #input = nn.functional.relu(self.fclayer1(input))
        #input = self.fclayer2(input)
        #return input

#Get cpu or gpu device for training.
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device')

model = Network().to(device)
if os.path.exists("model.pth"):
    model.load_state_dict(torch.load("model.pth"))
    print("Loaded model from model.pth")
else:
    print("No model found, creating new model")
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    print("Training model")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print("Finished training model")
            
def test(dataloader, model, loss_fn):
    print("Testing model")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!") 

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

[1631345700.0, '40.667202', '-73.8665']
(tensor([ 40.6672, -73.8665]), tensor(1.6313e+09))
12000
8000
20000
Feature batch shape: torch.Size([2, 2])
Labels batch shape: torch.Size([2])
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Using cpu device
tensor([[-2.3751,  1.4638, -0.5688, -1.5353,  2.4759]],
       grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[-0.1313, -0.1636],
        [-0.3491, -0.2541],
        [-0.2522, -0.3283],
        [ 0.1443, -0.3445],
        [-0.0434, -0.2677],
        [ 0.2278, -0.0868],
        [ 0.1491,  0.1825],
        [ 0.2925, -0.1352]], requires_grad=True), Parameter containing:
tensor([[ 0.2186,  0.0807, -0.2572, -0.0862,  0.2646,  0.2309, -0.0459, -0.1217],
        [-0.1495,  0.0509,  

TypeError: forward() missing 1 required positional argument: 'h0'

In [3]:

import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datetime
import sqlite3
import sys
import traceback
import numpy as np
import Data.database_handler as dbHandler
from torchvision import transforms, utils

#Connecting to the SQLite database
dbPath = r'Data\datasetNY.db'

class AccidentDataset(Dataset):
    def __init__(self, db_path, transform=None):
        self.coordinates = dbHandler.get_n_data_datetime_converted(db_path, 20000)
        self.coordinates = pd.DataFrame(self.coordinates, columns=['datetime', 'latitude', 'longitude'])
        self.transform = transform

    def __len__(self):
        return len(self.coordinates)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        datetime = self.coordinates.iloc[idx, 0:1]
        datetime = np.array([datetime]).squeeze()
        datetime = datetime.astype('float')
        datetime = torch.from_numpy(datetime).float()
        #datetime = datetime.reshape(1, -1)

        # generate random coordinates in our area with both lat and long

        # generate random number between 40.5 and 40.9 for lat
        rand1 = np.random.uniform(low=40.5, high=40.9, size=(1, 1))
        # generate random number between -74.3 and -73.7 for long
        rand2 = np.random.uniform(low=-74.3, high=-73.7, size=(1, 1))
        #combine both into a 2d array
        randomcoords = np.concatenate((rand1, rand2), axis=1).squeeze()
        #convert to tensor
        randomcoords = torch.from_numpy(randomcoords).float()
        randomcoords = randomcoords / 100

        coordinates = self.coordinates.iloc[idx, 1:3]
        coordinates = np.array([coordinates]).squeeze()
        coordinates = coordinates.astype('float')
        coordinates = torch.from_numpy(coordinates).float()
        coordinates = coordinates / 100
        sample = {'coordinates': coordinates, 'datetime' : datetime}

        if self.transform:
            sample = self.transform(sample)

        return randomcoords, coordinates

accident_dataset = AccidentDataset(dbPath)

#Create new array with 60% of the data
train_size = int(0.6 * len(accident_dataset))
test_size = len(accident_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(accident_dataset, [train_size, test_size])

print(len(train_dataset))
print(len(test_dataset))
print(len(accident_dataset))

#Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# define the class for multilinear regression
class Network(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self,x):
        y_pred = self.linear(x)
        return y_pred


# define the class for multilinear regression
# building the model object
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device')

model = Network(2, 2).to(device)

# define the loss function
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# define the training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    print("Training model")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #print(X)
        #print(y)
        # Compute prediction error
        pred = model(X)
        #print(pred)
        loss = loss_fn(pred, y)
        #print(loss)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(loss)

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print("Finished training model")

def test(dataloader, model, loss_fn):
    print("Testing model")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
# X = torch.tensor([[2.0, 3],[4.0, 5],[6.0, 7]])
# Y_pred = model(X)
# print(Y_pred)

[1631345700.0, '40.667202', '-73.8665']
12000
8000
20000
Using cuda device
Epoch 1
-------------------------------
12000
Training model
loss: 1.176899  [    2/12000]
loss: 0.835652  [  202/12000]
loss: 0.593157  [  402/12000]
loss: 0.421111  [  602/12000]
loss: 0.298545  [  802/12000]
loss: 0.211975  [ 1002/12000]
loss: 0.149956  [ 1202/12000]
loss: 0.106704  [ 1402/12000]
loss: 0.075718  [ 1602/12000]
loss: 0.053885  [ 1802/12000]
loss: 0.038226  [ 2002/12000]
loss: 0.026885  [ 2202/12000]
loss: 0.019190  [ 2402/12000]
loss: 0.013513  [ 2602/12000]
loss: 0.009427  [ 2802/12000]
loss: 0.006905  [ 3002/12000]
loss: 0.004910  [ 3202/12000]
loss: 0.003406  [ 3402/12000]
loss: 0.002405  [ 3602/12000]
loss: 0.001735  [ 3802/12000]
loss: 0.001218  [ 4002/12000]
loss: 0.000852  [ 4202/12000]
loss: 0.000637  [ 4402/12000]
loss: 0.000401  [ 4602/12000]
loss: 0.000315  [ 4802/12000]
loss: 0.000238  [ 5002/12000]
loss: 0.000161  [ 5202/12000]
loss: 0.000116  [ 5402/12000]
loss: 0.000064  [ 5602/1